In [2]:
from multiprocessing import Pool
import crawling as aw_crawl
import glob
import os
import pandas as pd
from itertools import product
from datetime import datetime as dt

# Noticias epidemiológicas

Aproximadamente, el código demora 2h 28 minutos en recolectar las noticias de un chunk de cuatro enfermedades desde enero del 2005 hasta diciembre de 2024.
* No hay mayores problemas con colocar fechas imposibles, salvo un mayor tiempo de procesamiento. El query en Google solo indicará que no se encontraron resultados y, dado nuestro algoritmo, ignorará estos enlaces.

Si bien se ejecutó en paralelo, el tiempo total de procesamiento equivale a 17h 16 minutos.

In [2]:
datecombo = list(product(range(2005,2025),range(1,13)))
with open('diseases.txt','r') as f:
    diseases = [dis[:-1] for dis in f.readlines()]
    
subset = diseases[:5]

for disease in subset:
    print(f'{disease}: {dt.now()}')
    
    for datetuple in datecombo:
        aw_crawl.google_crawl(disease,aw_crawl.resolve_date(datetuple))

In [3]:
all_news = pd.concat([pd.read_csv('urls/' + file) for file in os.listdir('urls')],axis=0,ignore_index=True)
print(f'Dimensiones: {all_news.shape}')
print(f'Duplicados: {all_news.duplicated(subset=['link']).sum(axis=0)}')
# all_news.drop_duplicates(subset=[])

Dimensiones: (9640, 3)
Duplicados: 1627


In [4]:
from tqdm import tqdm

tqdm.pandas()
corpus = all_news['link'].progress_apply(lambda x: pd.Series(aw_crawl.info_json(x)))
all_news = pd.concat([all_news,corpus],axis=1)
all_news['dups'] = all_news.duplicated(subset=['links'],keep=False)

all_news.to_excel('all_news.xlsx')

  1%|▏         | 133/9640 [01:58<54:06,  2.93it/s]  

# CDC MINSA

Inicialmente, tardó 10 horas (584 minutos y 46 segundos) en extraer todos los pdfs.

In [4]:
años = list(range(2005,2025))
with Pool(processes=6) as pool:
    pool.map(aw_crawl.cdc_pdfs,años)

In [16]:
all_pdfs = [file for file in glob.glob('pdfs/**/*',recursive=True) if not os.path.isdir(file)]
print(f'Cantidad de PDFs recolectados: {len(all_pdfs)}')

Cantidad de PDFs recolectados: 3840
